In [2]:
import pandas as pd
import glob
import os
from pathlib import Path

In [3]:
# function to summarize evaluation detail files for table in paper
def summarize_evaluation_reports_details(evaluation_reports_dir, filter_pattern="*details*.csv", output_filename="evaluation_summary.csv"):
    """
    Summarize evaluation detail files by calculating averages, sums, and counts across specified columns.
    Args:
        evaluation_reports_dir (str): Path to the evaluation_reports directory
        filter_pattern (str): Glob pattern to filter files (default: "*details*.csv")
        output_filename (str): Name for the output CSV file (saved in current directory)
    Returns:
        pd.DataFrame: Summary table with aggregated metrics
    """
    
    # Find all matching CSV files
    search_pattern = os.path.join(evaluation_reports_dir, filter_pattern)
    csv_files = glob.glob(search_pattern)
    
    if not csv_files:
        print(f"No files found matching pattern: {search_pattern}")
        return None
    
    print(f"Found {len(csv_files)} files matching pattern")
    
    # Define columns to aggregate and their operations
    avg_columns = ['score', 'task_classification_score']
    sum_columns = ['elapsed_time', 'total_tokens', 'total_cost', 'input_tokens', 'output_tokens']
    count_columns = ['question']  # Will count non-excluded rows
    
    summary_data = []
    
    for csv_file in csv_files:
        try:
            # Read the CSV file
            df = pd.read_csv(csv_file)
            
            # Filter out EXCLUDED rows
            df_filtered = df[df['score'] != 'EXCLUDED'].copy()
            
            if df_filtered.empty:
                print(f"Warning: No non-excluded rows in {os.path.basename(csv_file)}")
                continue
            
            # Convert numeric columns, handling empty strings and non-numeric values
            for col in avg_columns + sum_columns:
                if col in df_filtered.columns:
                    # Replace empty strings with NaN, then convert to numeric
                    df_filtered[col] = pd.to_numeric(df_filtered[col].replace('', pd.NA), errors='coerce')
            
            # Calculate aggregations
            row_data = {'filename': os.path.basename(csv_file)}
            
            # Average columns
            for col in avg_columns:
                if col in df_filtered.columns:
                    avg_val = df_filtered[col].mean()
                    row_data[f'average_{col}'] = avg_val if pd.notna(avg_val) else 0
                else:
                    row_data[f'average_{col}'] = 0
            
            # Sum columns
            for col in sum_columns:
                if col in df_filtered.columns:
                    sum_val = df_filtered[col].sum()
                    row_data[f'sum_{col}'] = sum_val if pd.notna(sum_val) else 0
                else:
                    row_data[f'sum_{col}'] = 0
            
            # Count columns (count of non-excluded questions)
            for col in count_columns:
                if col in df_filtered.columns:
                    row_data[f'count_{col}'] = len(df_filtered)
                else:
                    row_data[f'count_{col}'] = 0
            
            summary_data.append(row_data)
            print(f"Processed: {os.path.basename(csv_file)} ({len(df_filtered)} valid rows)")
            
        except Exception as e:
            print(f"Error processing {csv_file}: {str(e)}")
            continue
    
    if not summary_data:
        print("No data could be processed from the files")
        return None
    
    # Create summary DataFrame
    summary_df = pd.DataFrame(summary_data)
    
    # Reorder columns to have filename first, then aggregated metrics
    cols = ['filename']
    cols.extend([f'average_{col}' for col in avg_columns])
    cols.extend([f'sum_{col}' for col in sum_columns])
    cols.extend([f'count_{col}' for col in count_columns])
    
    summary_df = summary_df[cols]
    
    # Save to CSV in current directory
    output_path = os.path.join(os.getcwd(), output_filename)
    summary_df.to_csv(output_path, index=False, float_format='%.6f')
    
    print(f"\nSummary saved to: {output_path}")
    print(f"Summary shape: {summary_df.shape}")
    
    # Display the summary
    print("\nSUMMARY TABLE:")
    print("=" * 100)
    
    # Format display with better column width
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', 30)
    
    print(summary_df.to_string(index=False, float_format='%.6f'))
    
    return summary_df


In [4]:
summary = summarize_evaluation_reports_details(
        evaluation_reports_dir="../results/nba_finals/evaluation_reports",
        filter_pattern="*_agent_details_updated*.csv",
        output_filename="agent_updated_summary.csv"
)

Found 50 files matching pattern
Processed: claude-3-5-haiku-20241022_agent_details_updated.csv (441 valid rows)
Processed: claude-3-7-sonnet-20250219_agent_details_updated.csv (441 valid rows)
Processed: claude-opus-4-1-20250805_agent_details_updated.csv (441 valid rows)
Processed: claude-sonnet-4-20250514_agent_details_updated.csv (441 valid rows)
Processed: gemini-1.5-flash-8b_agent_details_updated.csv (441 valid rows)
Processed: gemini-1.5-flash_agent_details_updated.csv (441 valid rows)
Processed: gemini-2.0-flash_agent_details_updated.csv (441 valid rows)
Processed: gemini-2.5-flash-lite_agent_details_updated.csv (441 valid rows)
Processed: google_gemma-2-2b-it_agent_details_updated.csv (441 valid rows)
Processed: google_gemma-2-9b-it_agent_details_updated.csv (441 valid rows)
Processed: google_gemma-3-1b-it_agent_details_updated.csv (441 valid rows)
Processed: google_gemma-3-27b-it_agent_details_updated.csv (441 valid rows)
Processed: gpt-3.5-turbo_agent_details_updated.csv (441 